In [64]:
import numpy as np
import pandas as pd

In [65]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

In [66]:
train = train.drop(columns=['id'])
test = test.drop(columns=['id'])

In [67]:
train.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,target
0,0,0,6,1,0,0,0,0,7,0,...,0,0,0,0,0,0,2,0,0,Class_6
1,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,1,0,Class_6
2,0,0,0,0,0,1,0,3,0,0,...,0,0,0,0,1,0,0,0,0,Class_2
3,0,0,7,0,1,5,2,2,0,1,...,0,4,0,2,2,0,4,3,0,Class_8
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Class_2


In [68]:
# ランダムアンダーサンプリングやります
from imblearn.under_sampling import RandomUnderSampler

#ランダムアンダーサンプリング
rank_5 = train["target"].value_counts()[8]

rus = RandomUnderSampler(
    sampling_strategy={"Class_1": rank_5, "Class_2": rank_5,"Class_3": rank_5, "Class_4": rank_5,"Class_5": rank_5, "Class_6": rank_5,"Class_7": rank_5, "Class_8": rank_5,"Class_9": rank_5}, random_state=71
)




In [103]:
from sklearn import preprocessing
#ランダムアンダーサンプリング
def UnderSampleFunction(seed):
    print(seed)
    rank_5 = train["target"].value_counts()[8]

    rus = RandomUnderSampler(
        sampling_strategy={"Class_1": rank_5, "Class_2": rank_5,"Class_3": rank_5, "Class_4": rank_5,"Class_5": rank_5, "Class_6": rank_5,"Class_7": rank_5, "Class_8": rank_5,"Class_9": rank_5}, random_state=seed
    )

    train_x = train.drop(['target'], axis=1)
    train_y = train['target']

    test_x = test.copy()

    
    le = preprocessing.LabelEncoder()
    le.fit(train_y)
    train_y = le.transform(train_y)
    train_y = pd.Series(train_y)

    lgb_train = lgb.Dataset(tr_x, tr_y)
    lgb_eval = lgb.Dataset(va_x, va_y)

    return lgb_train, lgb_eval

In [70]:
import missingno as msno
# msno.matrix(df=train, figsize=(20,14), color=(0.5,0,0))

In [71]:
train_x = train.drop(['target'], axis=1)
train_y = train['target']

test_x = test.copy()

In [72]:
train_x.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74
0,0,0,6,1,0,0,0,0,7,0,...,3,0,0,0,0,0,0,2,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,1,0
2,0,0,0,0,0,1,0,3,0,0,...,8,0,0,0,0,1,0,0,0,0
3,0,0,7,0,1,5,2,2,0,1,...,0,0,4,0,2,2,0,4,3,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [73]:
train_y.head()

0    Class_6
1    Class_6
2    Class_2
3    Class_8
4    Class_2
Name: target, dtype: object

In [74]:
##train_x, train_y = rus.fit_resample(train_x, train_y)

In [75]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(train_y)
train_y = le.transform(train_y)
train_y = pd.Series(train_y)

In [76]:
train_y.head()

0    5
1    5
2    1
3    7
4    1
dtype: int64

In [77]:
import lightgbm as lgb
#import optuna.integration.lightgbm as lgb
from sklearn.metrics import log_loss

In [78]:
from sklearn.model_selection import train_test_split
tr_x, va_x, tr_y, va_y = train_test_split(train_x, train_y, test_size=0.2, shuffle=True, random_state=42, stratify=train_y) 


params = {'objective': 'multiclass',
                  #'random_state': 10,
                  'metric': 'multi_logloss',
             'num_class': 9,
             'force_row_wise' :True}

params2 = {'objective': 'multiclass',
 'metric': 'multi_logloss',
 #'boosting_type':"dart",
 'num_class': 9,
 'force_row_wise': True,
 'feature_pre_filter': False,
 'lambda_l1': 9.743033661523311,
 'lambda_l2': 2.709400532224499e-07,
 'num_leaves': 33,
 'feature_fraction': 0.4,
 'bagging_fraction': 1.0,
 'bagging_freq': 0,
 'min_child_samples': 100,
 'num_iterations': 50,
 'early_stopping_round': 5}


In [79]:
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit

def imbalanced_data_split(X, y, test_size=0.2):
    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
    for train_index, test_index in sss.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        return X_train, X_test, y_train, y_test

In [105]:
from imblearn.under_sampling import RandomUnderSampler

def bagging(seed):
    lgb_train, lgb_eval = UnderSampleFunction(seed)
    model_bagging = lgb.train(params2,
            lgb_train,
            verbose_eval=False,
            early_stopping_rounds=10,
            num_boost_round=10,
            valid_names=['train', 'valid'], 
            valid_sets=[lgb_train, lgb_eval])
    return model_bagging

In [81]:
def bagging2(seed):
    sampler = RandomUnderSampler(random_state=seed, replacement=True)
    X_resampled, y_resampled = sampler.fit_resample(X_train, y_train)
    X_train2, X_valid, y_train2, y_valid = imbalanced_data_split(X_resampled, y_resampled, test_size=0.2)
    model_bagging = lgbm_train(X_train2, X_valid, y_train2, y_valid, lgbm_params)
    return model_bagging

In [106]:
models = []

for i in range(10):
    models.append(bagging(i))

0
[LightGBM] [Info] Total Bins 3286
[LightGBM] [Info] Number of data points in the train set: 160000, number of used features: 75
[LightGBM] [Info] Start training from score -3.088122
[LightGBM] [Info] Start training from score -2.102454
[LightGBM] [Info] Start training from score -2.603859
[LightGBM] [Info] Start training from score -3.749957
[LightGBM] [Info] Start training from score -4.178678
[LightGBM] [Info] Start training from score -1.350710
[LightGBM] [Info] Start training from score -2.605804
[LightGBM] [Info] Start training from score -1.351627
[LightGBM] [Info] Start training from score -2.057974
1
[LightGBM] [Info] Total Bins 3286
[LightGBM] [Info] Number of data points in the train set: 160000, number of used features: 75
[LightGBM] [Info] Start training from score -3.088122
[LightGBM] [Info] Start training from score -2.102454
[LightGBM] [Info] Start training from score -2.603859
[LightGBM] [Info] Start training from score -3.749957
[LightGBM] [Info] Start training from 

In [102]:
best_params, history = {}, []
# train
pass
model = lgb.train(params2,
            lgb_train,
            verbose_eval=False,
            early_stopping_rounds=10,
            num_boost_round=100,
            valid_names=['train', 'valid'], 
            valid_sets=[lgb_train, lgb_eval])

/Users/tokimune-i/.pyenv/versions/anaconda3-2020.11/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/Users/tokimune-i/.pyenv/versions/anaconda3-2020.11/lib/python3.8/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
[LightGBM] [Info] Total Bins 3286
[LightGBM] [Info] Number of data points in the train set: 160000, number of used features: 75
[LightGBM] [Info] Start training from score -3.088122
[LightGBM] [Info] Start training from score -2.102454
[LightGBM] [Info] Start training from score -2.603859
[LightGBM] [Info] Start training from score -3.749957
[LightGBM] [Info] Start training from score -4.178678
[LightGBM] [Info] Start training from score -1.350710
[Lig

In [84]:
va_pred = model.predict(va_x)
score = log_loss(va_y, va_pred)
print(f'logloss: {score:.4f}')

logloss: 1.7482


In [85]:
for model in models:
    va_pred = model.predict(va_x)
    score = log_loss(va_y, va_pred)
    print(f'logloss: {score:.4f}')


logloss: 1.7482
logloss: 1.7482
logloss: 1.7482
logloss: 1.7482
logloss: 1.7482
logloss: 1.7482
logloss: 1.7482
logloss: 1.7482
logloss: 1.7482
logloss: 1.7482


In [96]:
pred_list = []
for i in models:
    j = i.predict(test_x)
    pred_list.append(j)




In [98]:
pred = np.mean(pred_list)
pred = np.array(pred_list).mean(axis=0)

In [87]:
#pred = model.predict(test_x)
#pred = np.array(pred_list).mean(axis=0)

In [88]:
model.params

{'objective': 'multiclass',
 'metric': 'multi_logloss',
 'num_class': 9,
 'force_row_wise': True,
 'feature_pre_filter': False,
 'lambda_l1': 9.743033661523311,
 'lambda_l2': 2.709400532224499e-07,
 'num_leaves': 33,
 'feature_fraction': 0.4,
 'bagging_fraction': 1.0,
 'bagging_freq': 0,
 'min_child_samples': 100,
 'num_iterations': 50,
 'early_stopping_round': 5}

In [97]:
submission = pd.DataFrame({'id': list(range(200000,300001))})
pred = pd.DataFrame(pred,columns =["Class_1","Class_2","Class_3","Class_4","Class_5","Class_6","Class_7","Class_8","Class_9"])
submission = pd.merge(submission,pred,left_index=True, right_index=True)
submission.to_csv('./sample_submission.csv', index=False)
print('csv output')

ValueError: DataFrame constructor not properly called!

In [90]:
train.describe()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.00000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.00000,200000.000000
mean,0.972710,1.168365,2.219325,2.296735,0.793530,1.431105,1.010695,0.673090,1.94398,1.719655,...,1.798040,0.508695,1.827300,0.910370,1.603585,1.219210,0.806895,1.282925,2.94021,0.632005
std,3.941836,3.993407,6.476570,7.551858,2.935785,5.162746,3.949231,2.234949,3.93133,5.916988,...,5.053014,1.867330,7.188924,3.835182,4.877679,4.826003,2.458741,4.261420,10.78465,3.925310
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
75%,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,2.00000,1.000000,...,1.000000,0.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.00000,0.000000
max,61.000000,51.000000,64.000000,70.000000,38.000000,76.000000,43.000000,30.000000,38.00000,72.000000,...,54.000000,24.000000,79.000000,55.000000,65.000000,67.000000,30.000000,61.000000,130.00000,52.000000


In [91]:
train['feature_0'].value_counts()

0     146389
1      24536
2      10197
3       6651
4       3580
5       3079
6       1751
7        671
8        349
34       144
29       140
20       130
40       129
32       113
61       110
9        106
37       102
56       100
42        95
47        88
27        87
12        87
48        83
24        81
39        78
31        76
15        76
30        73
10        71
21        70
16        70
22        68
14        67
43        66
25        66
19        64
11        63
28        63
13        63
17        57
26        56
23        55
Name: feature_0, dtype: int64

In [92]:
train_y.value_counts()

5    51811
7    51763
8    25542
1    24431
2    14798
6    14769
0     9118
3     4704
4     3064
dtype: int64

In [93]:
train_y.value_counts()

5    51811
7    51763
8    25542
1    24431
2    14798
6    14769
0     9118
3     4704
4     3064
dtype: int64